## Timetabling Program for St. Margaret's School (with Graph Colouring)

In [11]:
#import sys
#!{sys.executable} -m pip install ortools 

# import modules
import time
import numpy as np
import pandas as pd
import networkx as nx
import grinpy as gp
from random import random
from ortools.linear_solver import pywraplp

In [13]:
# Import Student Data
StudentInput = pd.ExcelFile("SMS Student Data.xlsx") 
StudentMatrix = pd.read_excel(StudentInput, 'Data') 
StudentInfo = StudentMatrix.values.tolist()
StudentMatrix[1:5]

,NAME,Grade,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10
1,S1,11,0,0,29,31,34,45,61,66,74,68
2,S2,11,16,0,29,34,47,55,61,71,74,0
3,S3,11,0,0,29,34,47,61,0,74,0,0
4,S4,11,0,29,34,46,52,63,68,75,0,45


In [3]:
# Import Course Data
CourseInput = pd.ExcelFile("SMS Course Data.xlsx") 
CourseMatrix = pd.read_excel(CourseInput, 'Data') 
CourseInfo = CourseMatrix.values.tolist()
CourseMatrix[1:5]

,Course Name,CourseID,Section,ABC,Teacher,FixedBlock,Core
1,Arts Education / ADST [Art Studio 11],3,1,NO,Louise,0,NO
2,Arts Education / ADST [Computer Programming 11],7,1,YES,William,0,NO
3,Arts Education / ADST [Culinary Arts 11],10,1,YES,Dana,0,NO
4,Arts Education / ADST [Culinary Arts 11],10,2,YES,Dana,0,NO


In [4]:
# Partition the offered courses into ShortCourses (ABC) and LongCourses (DEFGHI)
# We only consider the CourseID, not repeated sections of the course
ShortCourses=[]
LongCourses=[]
for m in range(len(CourseInfo)):
    CourseID = CourseInfo[m][1]
    if CourseInfo[m][2]==1:
        if CourseInfo[m][3]=="YES":
            ShortCourses.append(CourseID)
        else:
            LongCourses.append(CourseID)
            
# Partition the offered courses into OneSection and TwoSection
AllCourses=[CourseInfo[m][1] for m in range(len(CourseInfo))]
OneSection=[]
TwoSections=[]
for m in range(len(CourseInfo)):
    CourseID = CourseInfo[m][1]
    if CourseInfo[m][2]==2:
        TwoSections.append(CourseID)   
for CourseID in AllCourses:
    if not CourseID in TwoSections:
        OneSection.append(CourseID)
        
# Identify all Core Courses
CoreCourses=[]
for m in range(len(CourseInfo)):
    CourseID = CourseInfo[m][1]
    if CourseInfo[m][6]=="YES" and CourseInfo[m][2]==1:
        CoreCourses.append(CourseID)           
        
# Determine who is teaching each course
TeacherOfCourse = [0 for j in range(100)]
for j in range(len(CourseInfo)):
    CourseID = CourseInfo[j][1]
    TeacherName = CourseInfo[j][4]
    TeacherOfCourse[CourseID] = TeacherName

In [5]:
# Determine the preference coefficients based on the student data
# We have 10 points for a Core Course, 3 points for a Grade 12 elective
# and 1 point for a Grade 11 elective.

n = len(StudentInfo)
m = 100
P = np.zeros((n,m), dtype=int)
for i in range(n):
    for j in range(m):
        P[i,j]=-1

MaxScore=0
for i in range(n):
    for j in range(2,12):
        StudentGrade = StudentInfo[i][1]
        CoursePick = StudentInfo[i][j]
        if CoursePick > 0:
            if CoursePick in CoreCourses: P[i,CoursePick]=10
            else:
                if StudentGrade==12: P[i,CoursePick]=3
                if StudentGrade==11: P[i,CoursePick]=1

In [6]:
# Create Conflict Graph.  First start with the ShortCourses (ABC)

Vertices = list(set(ShortCourses).intersection(OneSection))
Vertices.sort()
v = len(Vertices)
M = np.zeros((v,v), dtype=int)
for x in range(v):
    for y in range(x+1,v):
        for i in range(n):
            if P[i,Vertices[x]]>0 and P[i,Vertices[y]]>0 :
                M[x,y] += min(P[i,Vertices[x]], P[i,Vertices[y]]) 
                

# Add 100 points for each pair of courses x and y taught by the same teacher

for x in range(v):
    for y in range(x+1,v):
        if TeacherOfCourse[Vertices[x]] == TeacherOfCourse[Vertices[y]]:
            M[x,y] += 100
            

# Find the chromatic number of the Conflict Graph.  If this number is above 3,
# eliminate the edges with weight 1.

G0=nx.Graph()
G0.add_nodes_from(Vertices)
for j1 in range(v):
    for j2 in range(j1+1, v):
        if M[j1,j2] > 0:
            G0.add_edge(Vertices[j1], Vertices[j2])
print("Chromatic number of G0 is", gp.chromatic_number(G0))

G1=nx.Graph()
G1.add_nodes_from(Vertices)
for j1 in range(v):
    for j2 in range(j1+1, v):
        if M[j1,j2] > 1:
            G1.add_edge(Vertices[j1], Vertices[j2])
print("Chromatic number of G1 is", gp.chromatic_number(G1))


Chromatic number of G0 is 5
Chromatic number of G1 is 3


In [7]:
flag=0
while flag==0:
    Colouring = nx.greedy_color(G1, strategy = 'random_sequential', interchange=True)
    Bundle1 = []
    Bundle2 = []
    Bundle3 = []
    for v in Vertices:
        if Colouring.get(v)==0: Bundle1.append(v)
        if Colouring.get(v)==1: Bundle2.append(v)
        if Colouring.get(v)==2: Bundle3.append(v)
        
    if len(Bundle1 + Bundle2 + Bundle3) == len(Vertices):
        flag=1

In [8]:
# Optimize short blocks

solver = pywraplp.Solver('St. Margarets School', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

start_time = time.time()

AllStudents = range(n)

Sections = [1,2]
ShortBlocks = [1,2,3]

# define boolean variables
x = {}
for s in Sections:
    for j in ShortCourses:
        for k in ShortBlocks:
            x[s,j,k] = solver.IntVar(0,1, 'x[%d,%d,%d]' % (s,j,k))

y = {}
for i in AllStudents:
    for j in ShortCourses:
        for k in ShortBlocks:
            y[i,j,k] = solver.IntVar(0,1, 'y[%d,%d,%d]' % (i,j,k)) 


# CONSTRAINT 1: For two-section courses, ensure each section is offered once
for j in set(ShortCourses).intersection(TwoSections):
    solver.Add(sum(x[1,j,k] for k in ShortBlocks) == 1)
    solver.Add(sum(x[2,j,k] for k in ShortBlocks) == 1)
    
# CONSTRAINT 2: For single-section courses, ensure x[2,j,k] is set to 0
for j in set(ShortCourses).intersection(OneSection):
    solver.Add(sum(x[1,j,k] for k in ShortBlocks) == 1)
    solver.Add(sum(x[2,j,k] for k in ShortBlocks) == 0)

# CONSTRAINT 3: Two sections of the same course can't be offered in the same block
for j in ShortCourses:
    for k in ShortBlocks:
        solver.Add(x[1,j,k] + x[2,j,k] <= 1)

# CONSTRAINT 4: At most eight classes per block
for k in ShortBlocks:
    solver.Add(sum(x[1,j,k] + x[2,j,k] for j in ShortCourses) <= 8)
        
# CONSTRAINT 5: No teacher can teach two classes in the same block
TeacherList = set([CourseInfo[z][4] for z in range(len(CourseInfo))])
for Teacher in TeacherList:
    ShortCourseList=[]
    for z in range(len(CourseInfo)):
        if CourseInfo[z][4]==Teacher and CourseInfo[z][1] in ShortCourses:
            ShortCourseList.append([CourseInfo[z][2],CourseInfo[z][1]])
    if len(ShortCourseList)>1:
        for k in ShortBlocks:
            solver.Add(sum(x[ShortCourseList[j][0],ShortCourseList[j][1],k] 
                           for j in range(len(ShortCourseList))) <= 1)
        
# CONSTRAINT 6: Ensure certain pre-defined courses are in fixed blocks
for z in range(len(CourseInfo)):
    if CourseInfo[z][5] in ShortBlocks:
        s = CourseInfo[z][2]
        j = CourseInfo[z][1]
        k = CourseInfo[z][5]
        solver.Add(x[s,j,k] == 1)
        
# CONSTRAINT 7: Each student takes at most one course per block
for k in ShortBlocks: 
    for i in AllStudents:
        solver.Add(sum(y[i,j,k] for j in ShortCourses) <= 1)

# CONSTRAINT 8: NO student can take the same course twice       
for j in ShortCourses:
    for i in AllStudents:
        solver.Add(sum(y[i,j,k] for k in ShortBlocks) <= 1)  

# CONSTRAINT 9: No student can take a course in a block when that course isn't offered
for i in AllStudents:
    for j in ShortCourses:
        for k in ShortBlocks:
            solver.Add(y[i,j,k] <= x[1,j,k]+x[2,j,k])
            
# CONSTRAINT 10: Every two-section course must have at most 18 students
for j in set(ShortCourses).intersection(TwoSections):
    for k in ShortBlocks:
            solver.Add(sum(y[i,j,k] for i in AllStudents) <= 18)
        
# CONSTRAINT 11: Do not assign course j to a student i if P[i,j]<0
for i in AllStudents:
    for j in ShortCourses:
        if P[i,j]<0:
            for k in ShortBlocks:
                solver.Add(y[i,j,k]==0)
                

# CONSTRAINT 12: Use graph theory to pre-bundle some one-section courses
for MySet in [Bundle1, Bundle2, Bundle3]:   
    for v1 in range(len(MySet)):
        for v2 in range(v1+1, len(MySet)):
            for k in ShortBlocks:
                solver.Add(x[1,MySet[v1],k]==x[1,MySet[v2],k])

    
solver.Maximize(solver.Sum(P[i,j]*y[i,j,k] for i in AllStudents for j in ShortCourses for k in ShortBlocks))
sol = solver.Solve()
print("")
print('Optimization Complete with Total Happiness Score of', round(solver.Objective().Value()))

# compute runtime
solving_time = time.time() - start_time

print('The code ran in', round(solving_time,1), 'seconds')



Optimization Complete with Total Happiness Score of 885
The code ran in 0.9 seconds


In [9]:
# Print the courses scheduled in each Short block and the number of
# students registered in each course.

ShortSchedule=[ [0,0,0] for i in AllStudents]
for i in AllStudents:
    for j in ShortCourses:
        for k in ShortBlocks:
            if y[i,j,k].solution_value()==1: 
                ShortSchedule[i][k-1]=j
                if j not in StudentInfo[i]:
                    print("ERROR", i,j,k)
                    
for k in ShortBlocks:
    print()
    print("Block", k, "courses are")

    for j in ShortCourses:
        if x[1,j,k].solution_value()==1:
            count=0
            for i in AllStudents:
                if ShortSchedule[i][k-1]==j:
                    count+=1
            print("Course", j, "Section 1 with", count, "students")
        if x[2,j,k].solution_value()==1:
            count=0
            for i in AllStudents:
                if ShortSchedule[i][k-1]==j:
                    count+=1
            print("Course", j, "Section 2 with", count, "students")
            
            
print()
print(Bundle1)
print(Bundle2)
print(Bundle3)


Block 1 courses are
Course 7 Section 1 with 3 students
Course 10 Section 2 with 0 students
Course 28 Section 1 with 18 students
Course 29 Section 2 with 17 students
Course 45 Section 1 with 12 students
Course 56 Section 1 with 1 students
Course 78 Section 1 with 4 students
Course 80 Section 1 with 3 students

Block 2 courses are
Course 28 Section 2 with 7 students
Course 29 Section 1 with 16 students
Course 30 Section 1 with 7 students
Course 75 Section 1 with 7 students
Course 79 Section 1 with 5 students

Block 3 courses are
Course 10 Section 1 with 3 students
Course 12 Section 1 with 1 students
Course 16 Section 1 with 1 students
Course 31 Section 1 with 14 students
Course 53 Section 1 with 1 students
Course 59 Section 1 with 4 students
Course 76 Section 1 with 10 students

[12, 16, 31, 53, 59, 76]
[7, 45, 56, 78, 80]
[30, 75, 79]


In [10]:
# Create Conflict Graph.  Now do the long courses (DEFGHI)

Vertices = list(set(LongCourses).intersection(OneSection))
Vertices.sort()
v = len(Vertices)
M = np.zeros((v,v), dtype=int)
for x in range(v):
    for y in range(x+1,v):
        for i in range(n):
            if P[i,Vertices[x]]>0 and P[i,Vertices[y]]>0 :
                M[x,y] += min(P[i,Vertices[x]],P[i,Vertices[y]]) 
                
# Add 100 points for each pair of courses x and y taught by the same teacher

for x in range(v):
    for y in range(x+1,v):
        if TeacherOfCourse[Vertices[x]] == TeacherOfCourse[Vertices[y]]:
            M[x,y] += 100
            

# Find the chromatic number of the Conflict Graph.  If this number is above 3,
# eliminate the edges with weight 1.

G0=nx.Graph()
G0.add_nodes_from(Vertices)
for j1 in range(v):
    for j2 in range(j1+1, v):
        if M[j1,j2] > 0:
            G0.add_edge(Vertices[j1], Vertices[j2])
print("Chromatic number of G0 is", gp.chromatic_number(G0))

G1=nx.Graph()
G1.add_nodes_from(Vertices)
for j1 in range(v):
    for j2 in range(j1+1, v):
        if M[j1,j2] > 1:
            G1.add_edge(Vertices[j1], Vertices[j2])
print("Chromatic number of G1 is", gp.chromatic_number(G1))

Chromatic number of G0 is 7
Chromatic number of G1 is 6


In [11]:
flag=0
while flag==0:
    Colouring = nx.greedy_color(G1, strategy = 'random_sequential', interchange=True)
    Bundle4 = []
    Bundle5 = []
    Bundle6 = []
    Bundle7 = []
    Bundle8 = []
    Bundle9 = []
    for v in Vertices:
        if Colouring.get(v)==0: Bundle4.append(v)
        if Colouring.get(v)==1: Bundle5.append(v)
        if Colouring.get(v)==2: Bundle6.append(v)
        if Colouring.get(v)==3: Bundle7.append(v)
        if Colouring.get(v)==4: Bundle8.append(v)
        if Colouring.get(v)==5: Bundle9.append(v)

    if len(Bundle4 + Bundle5 + Bundle6 + Bundle7 + Bundle8 + Bundle9) == len(Vertices):
        flag=1

In [12]:
# Optimize long blocks

solver = pywraplp.Solver('St. Margarets School', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

start_time = time.time()

AllStudents = range(n)

Sections = [1,2]
LongBlocks = [4,5,6,7,8,9]

# define boolean variables
x = {}
for s in Sections:
    for j in LongCourses:
        for k in LongBlocks:
            x[s,j,k] = solver.IntVar(0,1, 'x[%d,%d,%d]' % (s,j,k))

y = {}
for i in AllStudents:
    for j in LongCourses:
        for k in LongBlocks:
            y[i,j,k] = solver.IntVar(0,1, 'y[%d,%d,%d]' % (i,j,k)) 

            
# CONSTRAINT 1: For two-section courses, ensure each section is offered once
for j in set(LongCourses).intersection(TwoSections):
    solver.Add(sum(x[1,j,k] for k in LongBlocks) == 1)
    solver.Add(sum(x[2,j,k] for k in LongBlocks) == 1)
    
# CONSTRAINT 2: For single-section courses, ensure x[2,j,k] is set to 0
for j in set(LongCourses).intersection(OneSection):
    solver.Add(sum(x[1,j,k] for k in LongBlocks) == 1)
    solver.Add(sum(x[2,j,k] for k in LongBlocks) == 0)

# CONSTRAINT 3: Two sections of the same course can't be offered in the same block
for j in LongCourses:
    for k in LongBlocks:
        solver.Add(x[1,j,k] + x[2,j,k] <= 1)

# CONSTRAINT 4: At most five classes per block
for k in LongBlocks:
    solver.Add(sum(x[1,j,k] + x[2,j,k] for j in LongCourses) <= 5)
        
# CONSTRAINT 5: No teacher can teach two classes in the same block
TeacherList = set([CourseInfo[z][4] for z in range(len(CourseInfo))])
for Teacher in TeacherList:
    LongCourseList=[]
    for z in range(len(CourseInfo)):
        if CourseInfo[z][4]==Teacher and CourseInfo[z][1] in LongCourses:
            LongCourseList.append([CourseInfo[z][2],CourseInfo[z][1]])
    if len(LongCourseList)>1:
        for k in LongBlocks:
            solver.Add(sum(x[LongCourseList[j][0],LongCourseList[j][1],k] 
                           for j in range(len(LongCourseList))) <= 1)
        
# CONSTRAINT 6: Ensure certain pre-defined courses are in fixed blocks
for z in range(len(CourseInfo)):
    if CourseInfo[z][5] in LongBlocks:
        s = CourseInfo[z][2]
        j = CourseInfo[z][1]
        k = CourseInfo[z][5]
        solver.Add(x[s,j,k] == 1)
        
# CONSTRAINT 7: Each student takes at most one course per block
for k in LongBlocks: 
    for i in AllStudents:
        solver.Add(sum(y[i,j,k] for j in LongCourses) <= 1)

# CONSTRAINT 8: NO student can take the same course twice       
for j in LongCourses:
    for i in AllStudents:
        solver.Add(sum(y[i,j,k] for k in LongBlocks) <= 1)  

# CONSTRAINT 9: No student can take a course in a block when that course isn't offered
for i in AllStudents:
    for j in LongCourses:
        for k in LongBlocks:
            solver.Add(y[i,j,k] <= x[1,j,k]+x[2,j,k])
            
# CONSTRAINT 10: Every two-section course must have at most 18 students
for j in set(LongCourses).intersection(TwoSections):
    for k in LongBlocks:
            solver.Add(sum(y[i,j,k] for i in AllStudents) <= 18)
        
        
# CONSTRAINT 11: Do not assign course j to a student i if P[i,j]<0
for i in AllStudents:
    for j in LongCourses:
        if P[i,j]<0:
            for k in LongBlocks:
                solver.Add(y[i,j,k]==0)
               
            
# CONSTRAINT 12: Use graph theory to pre-bundle some one-section courses
for MySet in [Bundle4, Bundle5, Bundle6, Bundle7, Bundle8, Bundle9]:   
    for v1 in range(len(MySet)):
        for v2 in range(v1+1, len(MySet)):
            for k in LongBlocks:
                solver.Add(x[1,MySet[v1],k]==x[1,MySet[v2],k])

                
solver.Maximize(solver.Sum(P[i,j]*y[i,j,k] for i in AllStudents for j in LongCourses for k in LongBlocks))
sol = solver.Solve()
print("")
print('Optimization Complete with Total Happiness Score of', round(solver.Objective().Value()))

# compute runtime
solving_time = time.time() - start_time
print('The code ran in', round(solving_time,1), 'seconds')



Optimization Complete with Total Happiness Score of 1267
The code ran in 4.4 seconds


In [13]:
# Print the courses scheduled in each Long block and the number of
# students registered in each course.

LongSchedule=[ [0,0,0,0,0,0] for i in AllStudents]
for i in AllStudents:
    for j in LongCourses:
        for k in LongBlocks:
            if y[i,j,k].solution_value()==1: 
                LongSchedule[i][k-4]=j
                if j not in StudentInfo[i]:
                    print("ERROR", i,j,k)
                    
for k in LongBlocks:
    print()
    print("Block", k, "courses are")

    for j in LongCourses:
        if x[1,j,k].solution_value()==1:
            count=0
            for i in AllStudents:
                if LongSchedule[i][k-4]==j:
                    count+=1
            print("Course", j, "Section 1 with", count, "students")
        if x[2,j,k].solution_value()==1:
            count=0
            for i in AllStudents:
                if LongSchedule[i][k-4]==j:
                    count+=1
            print("Course", j, "Section 2 with", count, "students")
            
print()
print(Bundle4)
print(Bundle5)
print(Bundle6)
print(Bundle7)
print(Bundle8)
print(Bundle9)


Block 4 courses are
Course 61 Section 1 with 17 students
Course 66 Section 1 with 11 students
Course 81 Section 1 with 21 students

Block 5 courses are
Course 2 Section 1 with 4 students
Course 34 Section 2 with 14 students
Course 42 Section 1 with 12 students
Course 63 Section 1 with 11 students
Course 67 Section 1 with 6 students

Block 6 courses are
Course 3 Section 1 with 9 students
Course 36 Section 1 with 10 students
Course 52 Section 1 with 10 students
Course 71 Section 1 with 10 students
Course 72 Section 1 with 9 students

Block 7 courses are
Course 38 Section 1 with 6 students
Course 47 Section 2 with 10 students
Course 65 Section 1 with 12 students
Course 70 Section 1 with 4 students
Course 74 Section 1 with 5 students

Block 8 courses are
Course 34 Section 1 with 15 students
Course 46 Section 1 with 7 students
Course 55 Section 1 with 12 students
Course 67 Section 2 with 7 students
Course 68 Section 1 with 9 students

Block 9 courses are
Course 36 Section 2 with 15 student

In [14]:
# Print stats on the quality of our final timetable

CoreMax=0
Gr12Max=0
Gr11Max=0
CoreScore=0
Gr12Score=0
Gr11Score=0

for i in AllStudents:
    for j in range(m):
        if P[i,j]==10: 
            CoreMax+=1
            if j in ShortSchedule[i]: CoreScore+=1
            if j in LongSchedule[i]: CoreScore+=1
        if P[i,j]==3: 
            Gr12Max+=1
            if j in ShortSchedule[i]: Gr12Score+=1
            if j in LongSchedule[i]: Gr12Score+=1
        if P[i,j]==1: 
            Gr11Max+=1
            if j in ShortSchedule[i]: Gr11Score+=1
            if j in LongSchedule[i]: Gr11Score+=1
                
print("TOTAL: Students got into", CoreScore, "out of", CoreMax, "core courses")
print("Grade 12s got into", Gr12Score, "out of", Gr12Max, "elective courses")
print("Grade 11s got into", Gr11Score, "out of", Gr11Max, "elective courses")

TOTAL: Students got into 167 out of 167 core courses
Grade 12s got into 118 out of 129 elective courses
Grade 11s got into 128 out of 151 elective courses


In [15]:
# Print list of all students who didn't get into ALL of their desired courses
# and how many courses that missed getting into

for i in AllStudents:
    choices=0
    for j in range(2,12):
        if StudentInfo[i][j]>0:
            choices+=1
    for j in ShortSchedule[i]:
        if j>0:
            choices-=1
    for j in LongSchedule[i]:
        if j>0:
            choices-=1
    if choices>0:
        missedlist = set(StudentInfo[i]) - set(ShortSchedule[i]) - set(LongSchedule[i]) - {11,12} - {StudentInfo[i][0]}
        print("Student", i, "missed", choices, "-", ShortSchedule[i],LongSchedule[i], missedlist)

Student 4 missed 2 - [29, 75, 0] [0, 63, 52, 0, 34, 46] {68, 45}
Student 5 missed 1 - [80, 29, 0] [61, 34, 3, 0, 46, 66] {79}
Student 6 missed 1 - [80, 29, 0] [61, 34, 0, 70, 55, 66] {46}
Student 7 missed 1 - [45, 29, 0] [66, 34, 0, 74, 68, 0] {7}
Student 8 missed 2 - [45, 29, 0] [0, 34, 3, 74, 68, 66] {80, 71}
Student 9 missed 1 - [29, 0, 31] [61, 0, 71, 47, 34, 66] {76}
Student 11 missed 1 - [29, 79, 12] [61, 0, 3, 0, 34, 0] {52}
Student 12 missed 1 - [78, 29, 0] [66, 63, 52, 0, 34, 46] {71}
Student 13 missed 1 - [45, 29, 31] [61, 34, 0, 0, 68, 66] {55}
Student 16 missed 1 - [29, 0, 76] [66, 0, 52, 38, 0, 47] {16}
Student 17 missed 1 - [45, 29, 0] [0, 0, 52, 38, 68, 46] {78}
Student 19 missed 1 - [45, 29, 0] [61, 34, 71, 0, 68, 66] {52}
Student 20 missed 1 - [29, 0, 31] [66, 34, 3, 0, 0, 46] {53}
Student 22 missed 1 - [29, 75, 10] [66, 63, 52, 38, 46, 0] {71}
Student 24 missed 1 - [29, 0, 31] [61, 34, 71, 0, 46, 66] {59}
Student 26 missed 1 - [45, 29, 31] [66, 63, 3, 0, 34, 46] {68}
